In [3]:
import numpy as np

X = np.array([1, 2, 3, 4, 5])
y = np.array([1, 2, 3, 4, 5])

X_b = np.c_[np.ones((len(X), 1)), X]

#θ = (X^T * X)^(-1) * X^T * y
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

X_new = np.array([[0], [2], [3]])
X_new_b = np.c_[np.ones((len(X_new), 1)), X_new]
y_predict = X_new_b.dot(theta_best)

print("Predictions:", y_predict)

Predictions: [2.66453526e-15 2.00000000e+00 3.00000000e+00]


In [4]:
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def gradient_descent(X, y, learning_rate, iterations):
    m = len(y)
    X_b = np.c_[np.ones((m, 1)), X]
    theta = np.random.randn(X_b.shape[1], 1)
    
    for i in range(iterations):
        z = X_b.dot(theta)
        predictions = sigmoid(z)
        gradients = X_b.T.dot(predictions - y) / m
        theta -= learning_rate * gradients
    
    return theta

X = np.array([[1], [2], [3], [4]])
y = np.array([[0], [0], [1], [1]])

theta_best = gradient_descent(X, y, learning_rate=0.1, iterations=1000)
print("Best parameters:", theta_best)

X_new = np.array([[1], [3]])
X_new_b = np.c_[np.ones((len(X_new), 1)), X_new]
predictions = sigmoid(X_new_b.dot(theta_best))
print("Predictions:", predictions)

Best parameters: [[-5.81461674]
 [ 2.43270285]]
Predictions: [[0.03286551]
 [0.81509942]]


In [5]:
import numpy as np
from collections import Counter

# Distance function
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b)**2))

# K-NN classifier
def knn(X_train, y_train, X_test, k=3):
    y_pred = []
    for test_point in X_test:
        distances = [euclidean_distance(test_point, x) for x in X_train]
        k_indices = np.argsort(distances)[:k]
        k_nearest_labels = [y_train[i] for i in k_indices]
        majority_vote = Counter(k_nearest_labels).most_common(1)[0][0]
        y_pred.append(majority_vote)
    return y_pred

# Sample data
X_train = np.array([[1, 1], [2, 2], [3, 3], [6, 6], [7, 7]])
y_train = np.array([0, 0, 0, 1, 1])
X_test = np.array([[4, 4], [5, 5]])

# Run K-NN classifier
y_pred = knn(X_train, y_train, X_test, k=3)
print("Predicted labels:", y_pred)

Predicted labels: [0, 1]


In [6]:
import numpy as np

# Sample data (each row is a data point)
X = np.array([[2.5, 2.4], [0.5, 0.7], [2.2, 2.9], [1.9, 2.2], [3.1, 3.0]])

# Standardize data (zero mean and unit variance)
X_meaned = X - np.mean(X, axis=0)

# Covariance matrix
cov_matrix = np.cov(X_meaned.T)

# Eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

# Sort eigenvectors by eigenvalues in decreasing order
idx = eigenvalues.argsort()[::-1]
eigenvectors = eigenvectors[:, idx]

# Transform the data to the new basis
X_pca = X_meaned.dot(eigenvectors)

print("PCA transformed data:\n", X_pca)

PCA transformed data:
 [[ 0.44362444 -0.20099093]
 [-2.17719404 -0.05500992]
 [ 0.57071239  0.36808609]
 [-0.12902465  0.06747325]
 [ 1.29188186 -0.17955849]]


In [7]:
import numpy as np

# Euclidean distance
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b)**2))

# K-Means algorithm
def kmeans(X, k, iterations):
    # Randomly initialize centroids
    centroids = X[np.random.choice(X.shape[0], k, replace=False)]
    
    for _ in range(iterations):
        # Assign points to the nearest centroid
        clusters = [[] for _ in range(k)]
        for point in X:
            distances = [euclidean_distance(point, centroid) for centroid in centroids]
            cluster_index = np.argmin(distances)
            clusters[cluster_index].append(point)
        
        # Update centroids
        new_centroids = np.array([np.mean(cluster, axis=0) for cluster in clusters])
        if np.all(centroids == new_centroids):
            break
        centroids = new_centroids
    
    return centroids, clusters

# Sample data
X = np.array([[1, 2], [1.5, 1.8], [5, 8], [8, 8], [1, 0.6], [9, 11]])

# Run K-Means
centroids, clusters = kmeans(X, k=2, iterations=100)
print("Centroids:\n", centroids)

Centroids:
 [[7.33333333 9.        ]
 [1.16666667 1.46666667]]


In [8]:
import numpy as np

# Calculate probabilities
def calculate_probabilities(X, y):
    unique_classes = np.unique(y)
    probabilities = {}
    
    for label in unique_classes:
        X_class = X[y == label]
        probabilities[label] = {}
        probabilities[label]['prior'] = len(X_class) / len(y)
        probabilities[label]['mean'] = np.mean(X_class, axis=0)
        probabilities[label]['var'] = np.var(X_class, axis=0)
    
    return probabilities

# Gaussian probability
def gaussian_probability(x, mean, var):
    coeff = 1 / np.sqrt(2 * np.pi * var)
    exponent = np.exp(-(x - mean)**2 / (2 * var))
    return coeff * exponent

# Predict class label
def predict(X_test, probabilities):
    predictions = []
    for x in X_test:
        posteriors = {}
        for label, params in probabilities.items():
            prior = np.log(params['prior'])
            likelihood = np.sum(np.log(gaussian_probability(x, params['mean'], params['var'])))
            posteriors[label] = prior + likelihood
        predictions.append(max(posteriors, key=posteriors.get))
    return predictions

# Sample data
X_train = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]])
y_train = np.array([0, 0, 1, 1, 1])
X_test = np.array([[2.5, 3.5], [3.5, 4.5]])

# Run Naive Bayes classifier
probabilities = calculate_probabilities(X_train, y_train)
y_pred = predict(X_test, probabilities)
print("Predictions:", y_pred)

Predictions: [1, 1]


In [9]:
import numpy as np

# Hinge loss function for SVM
def hinge_loss(X, y, w, b, reg_strength):
    n_samples = X.shape[0]
    distances = 1 - y * (np.dot(X, w) + b)
    losses = np.maximum(0, distances)
    return reg_strength * (np.sum(losses) / n_samples)

# Gradient descent for SVM
def gradient_descent(X, y, w, b, learning_rate, iterations, reg_strength):
    for _ in range(iterations):
        for i, x_i in enumerate(X):
            condition = y[i] * (np.dot(X[i], w) + b) >= 1
            if condition:
                w -= learning_rate * (2 * reg_strength * w)
            else:
                w -= learning_rate * (2 * reg_strength * w - np.dot(X[i], y[i]))
                b -= learning_rate * y[i]
    return w, b

# Sample data
X = np.array([[1, 2], [2, 3], [3, 3], [4, 5], [5, 6]])
y = np.array([1, 1, -1, -1, -1])

# Parameters
w = np.zeros(X.shape[1])
b = 0
learning_rate = 0.001
iterations = 1000
reg_strength = 0.01

# Train SVM
w, b = gradient_descent(X, y, w, b, learning_rate, iterations, reg_strength)
print("Weight vector:", w)
print("Bias term:", b)

Weight vector: [-0.72999255  0.57392065]
Bias term: -0.9520000000000007
